In [3]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, deque
import pandas as pd
import urllib.parse

In [9]:
# Load all the data from the tsv files, skipping the headers and specifying column names
DATA_FOLDER = 'data/wikispeedia_paths-and-graph/'
articles = pd.read_csv(DATA_FOLDER + 'articles.tsv', sep='\t', skiprows=12, names=['article'])
categories = pd.read_csv(DATA_FOLDER + 'categories.tsv', sep='\t', skiprows=12, names=['article', 'category'])
links = pd.read_csv(DATA_FOLDER + 'links.tsv', sep='\t', skiprows=11, names=['linkSource', 'linkTarget'])
paths_finished = pd.read_csv(DATA_FOLDER + 'paths_finished.tsv', sep='\t', skiprows=15, names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'])
paths_unfinished = pd.read_csv(DATA_FOLDER + 'paths_unfinished.tsv', sep='\t', skiprows=16, names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'target', 'type'])
paths_openai = pd.read_csv('data/merged_file_final_openai.tsv', sep='\t', names=['path_id', 'steps', 'path'])

In [5]:
# Decode the URL-encoded article titles
articles = articles.map(urllib.parse.unquote)
categories = categories.map(urllib.parse.unquote)
links = links.map(urllib.parse.unquote)
paths_finished['path'] = paths_finished['path'].map(urllib.parse.unquote)
paths_unfinished['path'] = paths_unfinished['path'].map(urllib.parse.unquote)

In [6]:
# Prior click probability

# The probability to click on each of article a’s L_a outlinks is the same for all outlinks
# So we don’t need a third dimension to index the specific outlinks

# Actually, it doesn’t depend either on the goal, because our prior is that the links are clicked randomly

# Count the outlinks of each article to get the probability of clicking on any of them
out_degree = links.groupby('linkSource').size()
probs_prior = 1 / out_degree
# Create a Series indexed by 'linkSource' that gives a list of all the source’s outlinks
out_links = links.groupby('linkSource')['linkTarget'].agg(list)

In [ ]:
# N(A=a, G=g): the number of times 'a' was encountered on paths for which 'g' was the goal
count_goal_article = defaultdict(lambda: defaultdict(int))
# N(A’=a’, A=a, G=g): the number of times a’ was clicked in this situation
count_goal_article_article_clicked = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
# Filter out the paths that have only one article, because I have no clue what they mean
paths = paths_finished['path'][paths_finished['path'].apply(len) > 1]

# Create simplified paths by getting rid of the backtracking steps, and going straight
# where the player ended up going after backtracking
def straighten_path(path):
    stack = deque()
    for article in path:
        if article == '<':
            stack.pop()
        else:
            stack.append(article)
    return list(stack)

paths_no_backtrack = paths.apply(straighten_path)

# Count the occurrences of a, a’ and g along every path
for path in paths_no_backtrack:
    goal = path[-1]
    # Iterate through the path by getting each time one article and the one that was clicked from it.
    # It starts at (start_article, first_article_clicked) and ends with (before_last_article, goal).
    for article, article_clicked in zip(path, path[1:]):
        count_goal_article[goal][article] += 1
        count_goal_article_article_clicked[goal][article][article_clicked] += 1

In [ ]:
# N(A=a, G=g): the number of times 'a' was encountered on paths for which 'g' was the goal
count_goal_article = defaultdict(lambda: defaultdict(int))
# N(A’=a’, A=a, G=g): the number of times a’ was clicked in this situation
count_goal_article_article_clicked = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
# Filter out the paths that have only one article, because I have no clue what they mean
paths_openai = paths_openai['path'][paths_openai['path'].apply(len) > 1]


# Count the occurrences of a, a’ and g along every path
for path in paths_openai:
    goal = path[-1]
    # Iterate through the path by getting each time one article and the one that was clicked from it.
    # It starts at (start_article, first_article_clicked) and ends with (before_last_article, goal).
    for article, article_clicked in zip(path, path[1:]):
        count_goal_article[goal][article] += 1
        count_goal_article_article_clicked[goal][article][article_clicked] += 1

In [8]:
# Posterior click probabilities
probs_posterior = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
# alpha_ is the Dirichlet parameter representing the initial confidence in the uniform prior distribution
alpha_ = 0.1

for g, article_counts in count_goal_article.items():
    for a, count in article_counts.items():
        for a_ in out_links[a]:
            # Use the formula (1) from the Wikispeedia paper
            probs_posterior[g][a][a_] = (
                    (count_goal_article_article_clicked[g][a][a_] + alpha_)
                    /
                    (count_goal_article[g][a] + alpha_ * out_degree[a])
            )

KeyError: '1'

In [ ]:
def path_to_prior_entropy(path):
    """
    Turn a path into the prior entropy at each article along the path
    
    Parameters:
        path (array of strings): list of the titles of the articles along the path
        
    Returns:
        entropies (array of floats): list of the prior entropy values for each article along this path
    """
    # Skip the goal because the entropy is 0 at the goal
    return [-1 * out_degree[a] * probs_prior[a] * np.log(probs_prior[a]) for a in path[:-1]]

def path_to_posterior_entropy(path):
    """
    Turn a path into the posterior entropy at each article along the path
    
    Parameters:
        path (array of strings): list of the titles of the articles along the path
        
    Returns:
        entropies (array of floats): list of the posterior entropy values for each article along this path
    """
    g = path[-1]
    # Skip the goal because the entropy is 0 at the goal
    return [(-1 * sum([prob * np.log(prob) for prob in probs_posterior[g][a].values()])) for a in path[:-1]]

entropies_prior = paths_no_backtrack.apply(path_to_prior_entropy)
entropies_posterior = paths_no_backtrack.apply(path_to_posterior_entropy)

In [7]:
# Visualize the entropies like in Fig. 2 of the Wikispeedia paper
def plot_normalized_positions(series, graph_title, n_bins=7):
    """
    Create a bar plot of binned averages along the length of an array,
    but plotted along an x-axis normalized to [0,1].
    
    Parameters:
    series: pandas.Series where each element is an array of numbers
    graph_titre: string with the name of the quantity plotted
    n_bins: number of bins to divide the [0,1] interval into
    """
    # Create empty lists to store normalized positions and values
    all_positions = []
    all_values = []

    # Process each array in the series
    for arr in series:
        length = len(arr)
        # Create normalized positions for this array
        positions = np.linspace(0, 1, length)

        all_positions.extend(positions)
        all_values.extend(arr)

    # Create a DataFrame with the normalized positions and values
    df = pd.DataFrame({
        'position': all_positions,
        'value': all_values
    })

    # Create bins and calculate statistics for each bin
    df['bin'] = pd.cut(df['position'], bins=n_bins, labels=[f'{i/n_bins:.2f}-{(i+1)/n_bins:.2f}' for i in range(n_bins)])

    bin_stats = df.groupby('bin', observed=True).agg({
        'value': ['mean']
    }).reset_index()

    # Flatten the column names
    bin_stats.columns = ['bin', 'mean']

    # Create the plot
    plt.figure(figsize=(8, 3))
    sns.barplot(data=bin_stats, x='bin', y='mean')

    plt.title(f'Average {graph_title} along normalized path distance')
    plt.xlabel('Normalized distance along the path')
    plt.ylabel('Average bits of information')
    plt.xticks(rotation=45)

    return plt.gcf()

plot_normalized_positions(entropies_prior, 'prior entropy')
plt.show()
plot_normalized_positions(entropies_posterior, 'posterior entropy')
plt.show()

NameError: name 'entropies_prior' is not defined

In [ ]:
# Posterior click probabilities
probs_posterior = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
# alpha_ is the Dirichlet parameter representing the initial confidence in the uniform prior distribution
alpha_ = 0.1

for g, article_counts in count_goal_article.items():
    for a, count in article_counts.items():
        for a_ in out_links[a]:
            # Use the formula (1) from the Wikispeedia paper
            probs_posterior[g][a][a_] = (
                    (count_goal_article_article_clicked[g][a][a_] + alpha_)
                    /
                    (count_goal_article[g][a] + alpha_ * out_degree[a])
            )

In [ ]:
# Define a function to subtract two lists element-wise
def subtract_lists(list1, list2):
    return [a - b for a, b in zip(list1, list2)]

# Subtract posterior entropy to prior entropy element-wise in each path to obtain information gain
information_gain = entropies_prior.combine(entropies_posterior, subtract_lists)

In [ ]:
plot_normalized_positions(information_gain, 'information gain')
plt.show()